In [11]:
import numpy as np
import pandas as pd
import xarray as xr

from interpolation import depth_interp_spline

In [12]:
file_list = ['data/soo_list_%d.txt' % d for d in np.arange(1969, 2025)] 
file_list

['data/soo_list_1969.txt',
 'data/soo_list_1970.txt',
 'data/soo_list_1971.txt',
 'data/soo_list_1972.txt',
 'data/soo_list_1973.txt',
 'data/soo_list_1974.txt',
 'data/soo_list_1975.txt',
 'data/soo_list_1976.txt',
 'data/soo_list_1977.txt',
 'data/soo_list_1978.txt',
 'data/soo_list_1979.txt',
 'data/soo_list_1980.txt',
 'data/soo_list_1981.txt',
 'data/soo_list_1982.txt',
 'data/soo_list_1983.txt',
 'data/soo_list_1984.txt',
 'data/soo_list_1985.txt',
 'data/soo_list_1986.txt',
 'data/soo_list_1987.txt',
 'data/soo_list_1988.txt',
 'data/soo_list_1989.txt',
 'data/soo_list_1990.txt',
 'data/soo_list_1991.txt',
 'data/soo_list_1992.txt',
 'data/soo_list_1993.txt',
 'data/soo_list_1994.txt',
 'data/soo_list_1995.txt',
 'data/soo_list_1996.txt',
 'data/soo_list_1997.txt',
 'data/soo_list_1998.txt',
 'data/soo_list_1999.txt',
 'data/soo_list_2000.txt',
 'data/soo_list_2001.txt',
 'data/soo_list_2002.txt',
 'data/soo_list_2003.txt',
 'data/soo_list_2004.txt',
 'data/soo_list_2005.txt',
 

In [13]:
meta = pd.read_csv('data/meta.txt', delimiter='\t', header=0)

In [14]:
east_sea_meta= meta[(meta['Area'] == 'East Sea') & (meta['End date(YYYY-MM-DD)'].isna())].drop(columns=['Zooplankton sampling depth(m)'])

In [15]:
east_sea_meta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 0 to 148
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Area                    74 non-null     object 
 1   Line-Station            74 non-null     object 
 2   Begin date(YYYY-MM-DD)  70 non-null     object 
 3   End date(YYYY-MM-DD)    0 non-null      object 
 4   Latitude(˚N)            74 non-null     float64
 5   Longitude(˚E)           74 non-null     float64
 6   Bottom depth(m)         70 non-null     object 
dtypes: float64(2), object(5)
memory usage: 4.6+ KB


In [ ]:
ds = xr.Dataset()
for f in file_list:
    df = pd.read_csv(f, delimiter='\t', skiprows=1,
                     usecols=[1, 2, 3, 7, 8, 9],
                     names=['cruiseline', 'station', 'date', 'depth', 'temperature', 'qc'])
    df = df[df['qc'] == 1]
    df = df.drop(columns='qc')
    df['date'] = pd.to_datetime(df['date'])
    depth_new = np.array([0, 10, 20, 30, 50, 75, 100, 125, 150, 200, 250, 300, 400, 500])


KeyError: 'cruiseline'